In [ ]:
import pandas as pd 
import numpy as np 
import seaborn as sns

In [ ]:
base = pd.read_csv("Movies_new.csv")

# Understanding the dataset

In [ ]:
base.dtypes

Describe function show that the Budget and Revenue columns has 0 as a min values, however, the info() function showed that there isnt any NaN value

In [ ]:
base.describe()
base.info()
base.count()
base.isnull().sum()

This function replace all 0 value to NaN and then it is possible to fill this values with the median of the specific column

In [ ]:
replace = base.replace(0, np.nan, inplace=True)

In [ ]:
budget_median = base.Budget.median()
revenue_median = base.Revenue.median()

In [ ]:
base["Budget"].fillna(budget_median, inplace = True)
base["Revenue"].fillna(revenue_median, inplace = True)

It's possible to save another csv with all columns complete

In [8]:
base.to_csv("Movies_median.csv", index=False)

# Finding NaN values or 0 values

* The search for NaN values didn't show any results
* Using the isnull () + sum() functions we see that all atributes are related to an data. It's necessary to found the data with 0 as a value 

In [ ]:
base.isnull().sum().sum()